In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

### Linear mapping from input to hidden $\rightarrow$ hidden to output layer

In [53]:
n = 20  # number of samples
d = 2  # input space dimension (variables)
train_dim = 0.5
k = int(n * train_dim)
x_np = np.random.randn(n, 2)
# x_np = np.hstack((x_np, np.ones((n,1))))  # column relative to the constant term

# w_12 = np.array([[2., 3., 2.], [0.3, 1., 0.]])  # matrix of weights (random numbers)
w_12 = np.array([[1., 0.], [5., 1.]])
w_32 = np.array([10., 1.])
b_1 = np.array([0., 0.])
b_2 = np.array([0., 0])

# linear case
y_np = w_32.dot(np.dot(w_12, x_np.T) + b_1.reshape(d, 1)*np.ones((d, n))) + b_2.dot(np.ones((d, n))) # output vector 

In [27]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
W1 = tf.Variable(1e-4*tf.ones([2, 2]))
W2 = tf.Variable(1e-4*tf.ones([1, 2]))
# b1 = tf.Variable(tf.ones([2, 1]))
# b2 = tf.Variable(tf.ones([1, 2]))

linear_model = tf.matmul(W1, tf.transpose(x)) # + b1*tf.ones([d, k])
linear_model = tf.matmul(W2, linear_model) # + tf.matmul(b2, tf.ones([d, k]))
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
optimizer = tf.train.GradientDescentOptimizer(0.0001)
train = optimizer.minimize(loss)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(100000):
        sess.run(train, {x: x_np[0:k, :], y: y_np[0:k]})
    
    """curr_W1, curr_W2, curr_b1, curr_b2, curr_loss = sess.run([W1, W2, b1, b2, loss], {x: x_np[0:k, :], y: y_np[0:k]})
    print("W1: %s W2: %s b1: %s b2: %s loss: %s"%(curr_W1, curr_W2, curr_b1, curr_b2, curr_loss))
    """
    curr_W1, curr_W2, curr_loss = sess.run([W1, W2, loss], {x: x_np[0:k, :], y: y_np[0:k]})
    print("W1: %s W2: %s loss: %s"%(curr_W1, curr_W2, curr_loss))
    print(sess.run(loss, {x:x_np[k:, :], y:y_np[k:]}))

W1: [[ 2.73556352  0.18246363]
 [ 2.73556352  0.18246363]] W2: [[ 2.74139357  2.74139357]] loss: 3.28552e-07
5.85898e-07


### Reparametrization - Comparison with the pseudoinverse solution
We fitted the following form
$ f(x) = W_{32} W_{12} x $  
Let's call $W_{12} = F$ and $W_{32} = G$
$$ y = x_1(g_{1}f_{11} + g_{2}f_{21}) + x_2(g_{1}f_{12} + g_{2}f_{22}) + b_1\cdot g + \sum b_2$$


In [28]:
print("real - predicted")
print(w_32.dot(w_12[: ,0]), curr_W2.dot(curr_W1[:, 0]))
print(w_32.dot(w_12[: ,1]), curr_W2.dot(curr_W1[:, 1]))
# print(w_32.dot(b_1), curr_W2.dot(curr_b1))

alpha = np.dot(np.linalg.pinv(x_np[0:k,:].dot(x_np[0:k,:].T)), y_np[0:k])  # alpha=(XXT)^{\pseudoInv}Y
w = np.dot(x_np[0:k,:].T, alpha)  # w = XT alpha
print("pseudo-inverse solution", w)

real - predicted
(15.0, array([ 14.99851227], dtype=float32))
(1.0, array([ 1.00040925], dtype=float32))
('pseudo-inverse solution', array([ 15.,   1.]))


In [71]:
# solution for the single layer case - no regularization

n = 20  # number of samples
d = 2  # input space dimension (variables)
train_dim = 0.5
k = int(n * train_dim)
x_np = np.random.randn(n, 2)
w_12 = np.array([[1., 0.], [5., 1.]])
w_32 = np.array([10., 1.])
b_1 = np.array([0., 0.])
b_2 = np.array([0., 0])

# linear case
y_np = w_32.dot(np.dot(w_12, x_np.T) + b_1.reshape(d, 1)*np.ones((d, n))) + b_2.dot(np.ones((d, n))) # output vector 

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
init_value = tf.constant(x_np[0, :], dtype=tf.float32, shape=([d,1]))
W_single_layer = tf.Variable(init_value)  # vector of d dimensions #
# b_single_layer = tf.Variable(1e-4*tf.ones([1,1]))
single_layer_funct = tf.matmul(x, W_single_layer) # + b_single_layer
squared_deltas = tf.square(single_layer_funct - y)
loss = tf.reduce_sum(squared_deltas)
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train = optimizer.minimize(loss)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(10000):
        sess.run(train, {x: x_np[0:k, :], y: y_np[0:k]})
        print(sess.run(loss, {x:x_np[0:k, :], y: y_np[0:k]}))
    curr_W, curr_loss = sess.run([W_single_layer, loss], {x: x_np[0:k, :], y: y_np[0:k]})
    print("W: %s loss: %s"%(curr_W, curr_loss))
    print(sess.run(loss, {x:x_np[k:, :], y:y_np[k:]}))
    
alpha = np.dot(np.linalg.pinv(x_np[0:k,:].dot(x_np[0:k,:].T)), y_np[0:k])  # alpha=(XXT)^{\pseudoInv}Y
w = np.dot(x_np[0:k,:].T, alpha)  # w = XT alpha
print("pseudo-inverse solution", w)

4501.32
4451.91
4422.9
4405.83
4395.76
4389.81
4386.28
4384.2
4382.96
4382.22
4381.78
4381.52
4381.36
4381.27
4381.22
4381.18
4381.16
4381.15
4381.14
4381.14
4381.14
4381.14
4381.14
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
4381.13
43

**Notes**  
initialization of both parameters to zeros

n_samples=2, n_dimensions=2  
n_iters=5e5, learning_rate=1e-3, training_loss=2115.4

n_samples=500, n_dimensions=2  (many times, different n_iters, learning_rates)  
n_iters=5e4, learning_rate=5e-6, training_loss=nan  
n_iters=1e4, learning_rate=5e-3, training_loss=nan, everything set to 1  
n_iters=1e4, learning_rate=5e-2, training_loss=nan, everything set to 1  
n_iters=1e4, learning_rate=5e-5, training_loss=nan, everything set to 1